_Lambda School Data Science, Unit 2_
# Sprint Challenge: Predict Steph Curry's shots 🏀
For your Sprint Challenge, you'll use a dataset with all Steph Curry's NBA field goal attempts. 
(Regular season and playoff games, from October 28, 2009, through June 5, 2019.) 
You'll predict whether each shot was made, using information about the shot and the game. This is hard to predict! Try to get above 60% accuracy. The dataset was collected with the [nba_api](https://github.com/swar/nba_api) Python library.

In [0]:
%%capture
import sys
if 'google.colab' in sys.modules:
    # Install packages in Colab
    !pip install category_encoders==2.*
    !pip install pandas-profiling==2.*

In [141]:
# Read data
import pandas as pd
url = 'https://drive.google.com/uc?export=download&id=1fL7KPyxgGYfQDsuJoBWHIWwCAf-HTFpX'
df = pd.read_csv(url)
# Check data shape
assert df.shape == (13958, 20)

To demonstrate mastery on your Sprint Challenge, do all the required, numbered instructions in this notebook.
To earn a score of "3", also do all the stretch goals.
You are permitted and encouraged to do as much data exploration as you want.
**1. Begin with baselines for classification.** Your target to predict is `shot_made_flag`. What is your baseline accuracy, if you guessed the majority class for every prediction?
**2. Hold out your test set.** Use the 2018-19 season to test. NBA seasons begin in October and end in June. You'll know you've split the data correctly when your test set has 1,709 observations.
**3. Engineer new feature.** Engineer at least **1** new feature, from this list, or your own idea.
- **Homecourt Advantage**: Is the home team (`htm`) the Golden State Warriors (`GSW`) ?
- **Opponent**: Who is the other team playing the Golden State Warriors?
- **Seconds remaining in the period**: Combine minutes remaining with seconds remaining, to get the total number of seconds remaining in the period.
- **Seconds remaining in the game**: Combine period, and seconds remaining in the period, to get the total number of seconds remaining in the game. A basketball game has 4 periods, each 12 minutes long.
- **Made previous shot**: Was Steph Curry's previous shot successful?
**4. Decide how to validate** your model. Choose one of the following options. Any of these options are good. You are not graded on which you choose.
- **Train/validate/test split: train on the 2009-10 season through 2016-17 season, validate with the 2017-18 season.** You'll know you've split the data correctly when your train set has 11,081 observations, and your validation set has 1,168 observations.
- **Train/validate/test split: random 80/20%** train/validate split.
- **Cross-validation** with independent test set. You may use any scikit-learn cross-validation method.
**5.** Use a scikit-learn **pipeline** to **encode categoricals** and fit a **Decision Tree** or **Random Forest** model.
**6.** Get your model's **validation accuracy.** (Multiple times if you try multiple iterations.) 
**7.** Get your model's **test accuracy.** (One time, at the end.)
**8.** Given a **confusion matrix** for a hypothetical binary classification model, **calculate accuracy, precision, and recall.**
### Stretch Goals
- Engineer 4+ new features total, either from the list above, or your own ideas.
- Make 2+ visualizations to explore relationships between features and target.
- Optimize 3+ hyperparameters by trying 10+ "candidates" (possible combinations of hyperparameters). You can use `RandomizedSearchCV` or do it manually.
- Get and plot your model's feature importances.

## 1. Begin with baselines for classification. 
>Your target to predict is `shot_made_flag`. What would your baseline accuracy be, if you guessed the majority class for every prediction?

In [142]:
print(pd.value_counts(df.shot_made_flag,normalize=True))
print('curry miss 52% of his show, whereas he made 47.2 throughout oct 28 - June5')

0    0.527081
1    0.472919
Name: shot_made_flag, dtype: float64
curry miss 52% of his show, whereas he made 47.2 throughout oct 28 - June5


## 2. Hold out your test set.

>Use the 2018-19 season to test. NBA seasons begin in October and end in June. You'll know you've split the data correctly when your test set has 1,709 observations.

In [143]:
df['game_date']

0        2009-10-28
1        2009-10-28
2        2009-10-28
3        2009-10-28
4        2009-10-28
            ...    
13953    2019-06-05
13954    2019-06-05
13955    2019-06-05
13956    2019-06-05
13957    2019-06-05
Name: game_date, Length: 13958, dtype: object

In [144]:
df.head(3)
df['game_date'] = pd.to_datetime(df['game_date'])
start_date = '2018-10-15'
end_date = '2019-07-01'
mask = (df['game_date'] >= start_date) & (df['game_date'] <= end_date)
test_2018_2019 = df.loc[mask]
test_2018_2019.shape

(1709, 20)

## 3. Engineer new feature.

>Engineer at least **1** new feature, from this list, or your own idea.
>
>- **Homecourt Advantage**: Is the home team (`htm`) the Golden State Warriors (`GSW`) ?
>- **Opponent**: Who is the other team playing the Golden State Warriors?
>- **Seconds remaining in the period**: Combine minutes remaining with seconds remaining, to get the total number of seconds remaining in the period.
>- **Seconds remaining in the game**: Combine period, and seconds remaining in the period, to get the total number of seconds remaining in the game. A basketball game has 4 periods, each 12 minutes long.
>- **Made previous shot**: Was Steph Curry's previous shot successful?

In [145]:
df.head(3)
def engineer_features(df):
    #avoid copy warning
    df = df.copy()
    #Does GSW have homecourt advanture?
    df['HomeCourt'] = df['htm'] == 'GSW'
    #does curry clutching taken a shot under 20 second in very close game?
    df['clutch'] =( (df['minutes_remaining'] == 0) & (df['seconds_remaining'] <= 10) & 
                    (df['scoremargin_before_shot'] > -4) & (df['scoremargin_before_shot'] < 0) &
                    (df['period'] == 4)
                    )    
    return df
df = engineer_features(df)
test_2018_2019 = engineer_features(test_2018_2019)
df.head()

,game_id,game_event_id,player_name,period,minutes_remaining,seconds_remaining,action_type,shot_type,shot_zone_basic,shot_zone_area,...,loc_x,loc_y,shot_made_flag,game_date,htm,vtm,season_type,scoremargin_before_shot,HomeCourt,clutch
0,20900015,4,Stephen Curry,1,11,25,Jump Shot,3PT Field Goal,Above the Break 3,Right Side Center(RC),...,99,249,0,2009-10-28,GSW,HOU,Regular Season,2.0,True,False
1,20900015,17,Stephen Curry,1,9,31,Step Back Jump shot,2PT Field Goal,Mid-Range,Left Side Center(LC),...,-122,145,1,2009-10-28,GSW,HOU,Regular Season,0.0,True,False
2,20900015,53,Stephen Curry,1,6,2,Jump Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),...,-60,129,0,2009-10-28,GSW,HOU,Regular Season,-4.0,True,False
3,20900015,141,Stephen Curry,2,9,49,Jump Shot,2PT Field Goal,Mid-Range,Left Side(L),...,-172,82,0,2009-10-28,GSW,HOU,Regular Season,-4.0,True,False
4,20900015,249,Stephen Curry,2,2,19,Jump Shot,2PT Field Goal,Mid-Range,Left Side Center(LC),...,-68,148,0,2009-10-28,GSW,HOU,Regular Season,0.0,True,False


## **4. Decide how to validate** your model. 
>Choose one of the following options. Any of these options are good. You are not graded on which you choose.
>- **Train/validate/test split: train on the 2009-10 season through 2016-17 season, validate with the 2017-18 season.** You'll know you've split the data correctly when your train set has 11,081 observations, and your validation set has 1,168 observations.
>- **Train/validate/test split: random 80/20%** train/validate split.
>- **Cross-validation** with independent test set. You may use any scikit-learn cross-validation method.

In [146]:
df['game_date'] = pd.to_datetime(df['game_date'])
start_date_train = '2008-10-15'
end_date_train = '2017-07-01'
mask_09_10 = (df['game_date'] >= start_date_train) & (df['game_date'] <= end_date_train)
train_2009_2017 = df.loc[mask_09_10]
print(train_2009_2017.shape)

start_date_validate = '2017-10-15'
end_date_validate= '2018-07-01'
mask_17_18 = (df['game_date'] >= start_date_validate) & (df['game_date'] <= end_date_validate)
validate_2017_2018 = df.loc[mask_17_18]
train_2009_2017['shot_made_flag'] = train_2009_2017['shot_made_flag'].astype(str)
validate_2017_2018['shot_made_flag'] = validate_2017_2018['shot_made_flag'].astype(str)

(11081, 22)


c:\users\d3mona\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
c:\users\d3mona\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [179]:
test_2018_2019['shot_made_flag'] = test_2018_2019['shot_made_flag'].astype(str)

In [ ]:
#drop columns
target = 'shot_made_flag'
#I want these features to be tested upon
features_in = df.drop(columns=[target ,'game_date', 'player_name','game_id','game_event_id'], axis=1)
#rearrange to numeric value then catagory
numeric_features = features_in.select_dtypes(include='number').columns.tolist()
cardinality = features_in.select_dtypes(exclude='number').nunique()
features_cardinality = cardinality[cardinality <= 10].index.tolist()
features = numeric_features + features_cardinality

11081    1
11082    0
11083    0
11084    0
11085    1
11086    1
11087    0
11088    1
11089    0
11090    0
11091    0
11092    0
11093    1
11094    1
11095    1
11096    1
11097    0
11098    0
11099    0
11100    0
11101    1
11102    0
11103    0
11104    1
11105    1
11106    1
11107    0
11108    0
11109    1
11110    0
11111    1
11112    0
11113    0
11114    1
11115    0
11116    1
11117    1
11118    0
11119    0
11120    1
11121    1
11122    1
11123    0
11124    0
11125    0
11126    1
11127    0
11128    0
11129    1
11130    1
11131    1
11132    1
11133    0
11134    0
11135    0
Name: shot_made_flag, dtype: int64

In [180]:
#assign features to X and target to y
X_train_09_17 = train_2009_2017[features]
y_train_09_17 = train_2009_2017[target]

X_Validate_17_18 = validate_2017_2018[features]
y_validate_17_18 = validate_2017_2018[target]

X_test = test_2018_2019[features]
y_test = test_2018_2019[target]

In [149]:
y_train_09_17.dtypes

dtype('O')

## 5. Use a scikit-learn pipeline to encode categoricals and fit a Decision Tree or Random Forest model.

In [128]:
from scipy.stats import randint,uniform
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
import category_encoders as ce
from sklearn.pipeline import make_pipeline

pipeline_Ordinary_RandomForestClassifier = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(strategy='mean'),
    RandomForestClassifier(n_estimators=6, random_state=42,n_jobs=-1)
)
Parameter_distrubution_Ordinary_randomForestClassifer = {
    'simpleimputer__strategy' : 'mean',
    'randomforestclassifier__n_estimators': 100,
    'randomforestclassifier__max_depth': 5,
}

search_Ordinary_randomForestClassifier = RandomizedSearchCV(
    pipeline_Ordinary_RandomForestClassifier,
    param_distributions=Parameter_distrubution_Ordinary_randomForestClassifer,
    n_iter=5,
    cv=5,
    scoring='accuracy',
    verbose=10,
    return_train_score=True,
    n_jobs=-1
)

In [150]:
# <editor-fold desc="###################Ordinal RandomForestclassifier,  randomizedSerachRv###################################">
pipeline = make_pipeline(
    ce.OrdinalEncoder(), 
    SimpleImputer(strategy='mean'), 
    RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
)

param_dist ={
    'simpleimputer__strategy': ['mean'],
    'randomforestclassifier__n_estimators': randint(50,500),
    'randomforestclassifier__max_depth': [5,10,15,None],
    'randomforestclassifier__max_features': uniform(0,1),   
}

search_RFClassifer = RandomizedSearchCV(
    pipeline,
    param_distributions=param_dist,
    n_iter=5,
    cv=3,
    verbose=10,
    scoring='accuracy',
    return_train_score=True,
    n_jobs=-1
)
# </editor-fold>

In [151]:
search_RFClassifer.fit(X_train_09_17,y_train_09_17)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of  15 | elapsed:   10.1s remaining:  1.1min
[Parallel(n_jobs=-1)]: Done   4 out of  15 | elapsed:   10.5s remaining:   29.0s
[Parallel(n_jobs=-1)]: Done   6 out of  15 | elapsed:   10.5s remaining:   15.8s
[Parallel(n_jobs=-1)]: Done   8 out of  15 | elapsed:   10.6s remaining:    9.2s
[Parallel(n_jobs=-1)]: Done  10 out of  15 | elapsed:   10.8s remaining:    5.3s
[Parallel(n_jobs=-1)]: Done  12 out of  15 | elapsed:   12.8s remaining:    3.1s
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:   13.7s finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=Pipeline(memory=None,
                                      steps=[('ordinalencoder',
                                              OrdinalEncoder(cols=None,
                                                             drop_invariant=False,
                                                             handle_missing='value',
                                                             handle_unknown='value',
                                                             mapping=None,
                                                             return_df=True,
                                                             verbose=0)),
                                             ('simpleimputer',
                                              SimpleImputer(add_indicator=False,
                                                            copy=True,
                                                            f

## 6.Get your model's validation accuracy
> (Multiple times if you try multiple iterations.)

In [152]:
score = search_RFClassifer.score(X_Validate_17_18,y_validate_17_18)
print('accuracy validation is', score)

accuracy validation is 0.5659246575342466


## 7. Get your model's test accuracy
> (One time, at the end.)

,period,minutes_remaining,seconds_remaining,shot_distance,loc_x,loc_y,scoremargin_before_shot,shot_type,shot_zone_basic,shot_zone_area,shot_zone_range,season_type,HomeCourt,clutch
12249,1,11,31,24,226,90,0.0,3PT Field Goal,Above the Break 3,Right Side Center(RC),24+ ft.,Regular Season,True,False
12250,1,7,57,6,-2,63,4.0,2PT Field Goal,In The Paint (Non-RA),Center(C),Less Than 8 ft.,Regular Season,True,False
12251,1,3,46,2,5,29,4.0,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,Regular Season,True,False
12252,1,2,58,1,-6,18,7.0,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,Regular Season,True,False
12253,1,2,49,22,-228,8,7.0,3PT Field Goal,Left Corner 3,Left Side(L),24+ ft.,Regular Season,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13953,4,8,1,12,3,120,-13.0,2PT Field Goal,In The Paint (Non-RA),Center(C),8-16 ft.,Playoffs,True,False
13954,4,7,16,11,114,-5,-14.0,2PT Field Goal,Mid-Range,Right Side(R),8-16 ft.,Playoffs,True,False
13955,4,5,27,26,-217,149,-17.0,3PT Field Goal,Above the Break 3,Left Side Center(LC),24+ ft.,Playoffs,True,False
13956,4,4,50,7,59,49,-16.0,2PT Field Goal,In The Paint (Non-RA),Center(C),Less Than 8 ft.,Playoffs,True,False


In [181]:
from sklearn.metrics import accuracy_score
y_pred_RFClassifer = search_RFClassifer.predict(X_test)
accuracy= accuracy_score(y_test, y_pred_RFClassifer)
print('test accuracy score is', accuracy)

test accuracy score is 0.5728496196606202


## 8. Given a confusion matrix, calculate accuracy, precision, and recall.

Imagine this is the confusion matrix for a binary classification model. Use the confusion matrix to calculate the model's accuracy, precision, and recall.

<table>
  <tr>
    <td colspan="2" rowspan="2"></td>
    <td colspan="2">Predicted</td>
  </tr>
  <tr>
    <td>Negative</td>
    <td>Positive</td>
  </tr>
  <tr>
    <td rowspan="2">Actual</td>
    <td>Negative</td>
    <td style="border: solid">85</td>
    <td style="border: solid">58</td>
  </tr>
  <tr>
    <td>Positive</td>
    <td style="border: solid">8</td>
    <td style="border: solid"> 36</td>
  </tr>
</table>

### Calculate accuracy 

In [167]:
(85+36)/(85+58+8+36)


0.6470588235294118

### Calculate precision

In [169]:
(85)/(85+6)


0.9340659340659341

### Calculate recall

In [172]:
85/(85+8)

0.9139784946236559